In [7]:
import pandas as pd

url="https://raw.githubusercontent.com/lichengrui/glassdoor_scraping/main/data.csv"

df =pd.read_csv(url)

cols = df.columns.tolist()

df = df.drop(columns=cols[0])

cols = df.columns.tolist()

df = df.dropna(subset=cols, how='all')

display(df)

,rating,employLengthStatus,title,date,jobTitle,prosWords,conwords,adviceWords,responsewords
1,5.0,"Current Employee, less than 1 year",Global Business with Great People and Highly C...,"Nov 9, 2022",Engineering Director,Highly global teams with many talents and pers...,Onboarding process has been a bit messy. Lots ...,Maintain the focus on inclusion and open commu...,NaN
2,5.0,"Current Employee, less than 1 year",Great Company,"Jul 27, 2023","Sr Appl/Sys Sales Engineer in Dallas, TX",I like how everyone is very helpful.,I have no cons at this time.,NaN,NaN
3,5.0,"Current Employee, less than 1 year",新人入職,"Jul 10, 2023",Project Engr I,My experience joining Honeywell has been incre...,1. Continuous growth: As I'm still in the earl...,I suggest organizing regular cross-departmenta...,NaN
4,5.0,"Current Employee, less than 1 year",Glassdoor,"Jul 11, 2023","Feild Service Tech in Grand Rapids, MI",I like it here at Honeywell because of the amo...,The sheer size of the company makes It hard to...,The management I work under is spread thin. Bu...,NaN
5,4.0,"Current Employee, more than 1 year",Great Place for The Right People!,"Jul 7, 2023",Anonymous Employee,Honeywell is an excellent place to work for th...,*Advancement always requires relocation *Fast ...,"Keep working on succession, learning/training ...",NaN
...,...,...,...,...,...,...,...,...,...
17646,5.0,"Former Employee, more than 1 year",Best Work Life Balance,"Jun 27, 2023",Finance System Analyst,Along with great packages we can have a good w...,Job Security is a big concern in the company,NaN,NaN
17647,4.0,"Current Employee, less than 1 year",Big Company,"Jun 22, 2023","Field Service Engineer in Houston, TX","Training, Zero-Micro Management. Diverse culture.",Too big for it's own good. You are a tad-pole ...,NaN,NaN
17648,5.0,Former Employee,financial,"Jul 5, 2023","Buyer in Aguadilla, PR","Flexible, a lot of learning, many contacts","Overtime, many projects and tight deadlines.",NaN,NaN
17649,5.0,"Current Employee, less than 1 year",First Month Experience,"Jun 28, 2023",Anonymous Employee,Accepting environment Meaningful work Free dri...,Fast pace working environment Can have a lot o...,NaN,NaN


In [9]:
#! pip install gensim

In [10]:

# imports
import numpy as np

import re
from nltk.corpus import stopwords
import nltk
import gensim

from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models

In [11]:
# Preprocess the sentences - tokenize and remove stop words
def preprocess_sentence(sentence):
    return [token for token in simple_preprocess(sentence) if token not in STOPWORDS]

df['processed_conwords'] = df['conwords'].apply(preprocess_sentence)

# Create a dictionary and a corpus (bag-of-words representation)
dictionary = corpora.Dictionary(df['processed_conwords'])
corpus = [dictionary.doc2bow(sentence) for sentence in df['processed_conwords']]

In [13]:

# Build the TF-IDF model from the corpus
tfidf_model = models.TfidfModel(corpus)
corpus_tfidf = tfidf_model[corpus]

# Build the LSI model using the TF-IDF corpus
num_topics = 10 
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics)

# Print the topics and their top keywords
for idx, topic in lsi_model.print_topics(-1):
    print(f"Topic #{idx}: {topic}")

   
def get_topic(processed_document):
    bow_vector = dictionary.doc2bow(processed_document)
    lsi_vector = lsi_model[tfidf_model[bow_vector]]
    if lsi_vector:  # Check if the vector is not empty
        dominant_topic = max(lsi_vector, key=lambda x: x[1])[0]
        return dominant_topic
    else:
        return -1  # Indicate that no dominant topic was found


df['topic'] = df['processed_conwords'].apply(get_topic)

# Print the DataFrame with topics
display(df)    

Topic #0: 0.975*"cons" + 0.101*"company" + 0.097*"good" + 0.079*"think" + 0.065*"honeywell" + 0.059*"far" + 0.042*"time" + 0.041*"work" + 0.040*"till" + 0.037*"working"
Topic #1: -0.492*"work" + -0.415*"balance" + -0.409*"life" + -0.249*"good" + -0.152*"company" + -0.144*"management" + 0.127*"cons" + -0.121*"benefits" + -0.111*"bad" + -0.103*"poor"
Topic #2: -0.981*"think" + 0.088*"cons" + 0.080*"balance" + 0.077*"life" + -0.053*"moment" + 0.047*"work" + -0.030*"continue" + -0.029*"reading" + -0.027*"management" + -0.026*"company"
Topic #3: -0.413*"balance" + -0.400*"life" + 0.369*"good" + 0.273*"company" + -0.226*"work" + 0.192*"growth" + 0.164*"management" + -0.158*"think" + 0.154*"benefits" + 0.148*"slow"
Topic #4: -0.837*"good" + 0.209*"growth" + 0.156*"slow" + -0.118*"far" + 0.114*"time" + 0.108*"career" + 0.102*"management" + 0.093*"continue" + 0.091*"reading" + 0.083*"long"
Topic #5: 0.558*"growth" + 0.383*"slow" + 0.268*"career" + -0.244*"bad" + -0.222*"company" + 0.207*"salary

,rating,employLengthStatus,title,date,jobTitle,prosWords,conwords,adviceWords,responsewords,processed_conwords,topic
1,5.0,"Current Employee, less than 1 year",Global Business with Great People and Highly C...,2022-11-09,Engineering Director,Highly global teams with many talents and pers...,Onboarding process has been a bit messy. Lots ...,Maintain the focus on inclusion and open commu...,NaN,"[onboarding, process, bit, messy, lots, self, ...",3
2,5.0,"Current Employee, less than 1 year",Great Company,2023-07-27,"Sr Appl/Sys Sales Engineer in Dallas, TX",I like how everyone is very helpful.,I have no cons at this time.,NaN,NaN,"[cons, time]",0
3,5.0,"Current Employee, less than 1 year",新人入職,2023-07-10,Project Engr I,My experience joining Honeywell has been incre...,1. Continuous growth: As I'm still in the earl...,I suggest organizing regular cross-departmenta...,NaN,"[continuous, growth, early, stages, onboarding...",3
4,5.0,"Current Employee, less than 1 year",Glassdoor,2023-07-11,"Feild Service Tech in Grand Rapids, MI",I like it here at Honeywell because of the amo...,The sheer size of the company makes It hard to...,The management I work under is spread thin. Bu...,NaN,"[sheer, size, company, makes, hard, salve, pro...",3
5,4.0,"Current Employee, more than 1 year",Great Place for The Right People!,2023-07-07,Anonymous Employee,Honeywell is an excellent place to work for th...,*Advancement always requires relocation *Fast ...,"Keep working on succession, learning/training ...",NaN,"[advancement, requires, relocation, fast, pace...",4
...,...,...,...,...,...,...,...,...,...,...,...
17646,5.0,"Former Employee, more than 1 year",Best Work Life Balance,2023-06-27,Finance System Analyst,Along with great packages we can have a good w...,Job Security is a big concern in the company,NaN,NaN,"[job, security, big, concern, company]",9
17647,4.0,"Current Employee, less than 1 year",Big Company,2023-06-22,"Field Service Engineer in Houston, TX","Training, Zero-Micro Management. Diverse culture.",Too big for it's own good. You are a tad-pole ...,NaN,NaN,"[big, good, tad, pole, ocean, lot, moving, par...",3
17648,5.0,Former Employee,financial,2023-07-05,"Buyer in Aguadilla, PR","Flexible, a lot of learning, many contacts","Overtime, many projects and tight deadlines.",NaN,NaN,"[overtime, projects, tight, deadlines]",4
17649,5.0,"Current Employee, less than 1 year",First Month Experience,2023-06-28,Anonymous Employee,Accepting environment Meaningful work Free dri...,Fast pace working environment Can have a lot o...,NaN,NaN,"[fast, pace, working, environment, lot, meetin...",7


In [14]:
df["topic"].value_counts()

 3    7300
 7    2005
 4    1605
 5    1315
 0    1155
 9    1071
 6     496
 8     470
 2     339
-1     139
Name: topic, dtype: int64